In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from processing_data import *

In [2]:
file_path = "C:/Users\Administrator\SIB\model/asdf_single_log.csv"

x_names = ['L%d%c' % (i, c) for i in range(21) for c in ['x', 'y', 'z']]
y_names = ['a', 's', 'd', 'f']

col_names = x_names + y_names

df = pd.read_csv(file_path,names=col_names)

In [3]:
#df = df.iloc[df.index[:340]]

In [4]:
p_df = df.copy()

cut_df = cut_outlier(df, x_names)
# p_df = preprocess(df,x_names, gamma=0.4)

In [5]:
gamma = 0.4
gamma_df = gamma_smoothing(cut_df, gamma, x_names)

C:\Users\Administrator\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [8]:
minmax_df = MinMaxScaler(gamma_df, x_names)

ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
x, y = df[x_names], df[y_names]

In [ ]:
train_test_ratio = 0.90

train_num = int(len(df) * train_test_ratio)

x_train, x_test = x[:train_num], x[train_num:]
y_train, y_test = y[:train_num], y[train_num:]

x_train_numpy = x_train.to_numpy()
x_test_numpy = x_test.to_numpy()
y_train_numpy = y_train.to_numpy()
y_test_numpy = y_test.to_numpy()

In [ ]:
x_train_t = x_train_numpy.reshape(x_train_numpy.shape[0],63,1)
x_test_t = x_test_numpy.reshape(x_test_numpy.shape[0],63,1)

In [ ]:
model = Sequential()

model.add(LSTM(50, input_shape=(63,1)))
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x_train_t, y_train, epochs=20)

In [ ]:
model.evaluate(x_test_t, y_test, verbose=2)

In [ ]:
predict = model.predict(x_test_t)

In [ ]:
size = len(y_test)

plt.plot(range(size), y_test['None'], 'k--')
plt.plot(range(size), predict[:,4],'k')

In [ ]:
plt.plot(range(size), y_test['a'], 'r--')
plt.plot(range(size), predict[:,0],'r')

In [ ]:
plt.plot(range(size), y_test['s'], 'g--')
plt.plot(range(size), predict[:,1],'g')

In [ ]:
plt.plot(range(size), y_test['d'], 'b--')
plt.plot(range(size), predict[:,2],'b')

In [ ]:
plt.plot(range(size), y_test['f'], 'c--')
plt.plot(range(size), predict[:,3],'c')